In [15]:
# passed on Python 3.10.7@Windows 11
# DO NOT RUN ON MACOS!!!!!!!!!
# np.linalg.dev(S_w) = -4e+192 on Windows but 0 on MacOS

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [16]:
df = pd.read_csv('data1forEx1to4/train1_icu_data.csv')
# mm = MinMaxScaler()
mm = StandardScaler()
df = pd.DataFrame(mm.fit_transform(df), columns=df.columns)
df_label = pd.read_csv('data1forEx1to4/train1_icu_label.csv')
df['label'] = df_label['hospital_death']

In [17]:
cluster_1 = df[df['label'] == 1].values[:, :-1]
cluster_0 = df[df['label'] == 0].values[:, :-1]

In [18]:
m_1 = np.mean(cluster_1, axis=0).reshape(-1, 1)
m_0 = np.mean(cluster_0, axis=0).reshape(-1, 1)

In [19]:
S_w = np.zeros((df.shape[1] - 1, df.shape[1] - 1))
for i in range(cluster_1.shape[0]):
    S_w += (cluster_1[i].reshape(-1, 1) - m_1) @ (cluster_1[i].reshape(-1, 1) - m_1).T
for i in range(cluster_0.shape[0]):
    S_w += (cluster_0[i].reshape(-1, 1) - m_0) @ (cluster_0[i].reshape(-1, 1) - m_0).T

In [20]:
w = np.linalg.inv(S_w) @ (m_1 - m_0)
w_0 = -0.5 * (w.T @ m_1 + w.T @ m_0)

In [21]:
def verify(x):
    return 1 if x > 0 else 0

result = np.vectorize(verify)(df.values[:, :-1] @ w + w_0[0][0])

In [22]:
check = result.reshape(1, -1)[0] == df['label'].values
check.sum() / check.shape[0]

0.541